# Using Custom Unit and Property Models

This notebook will demonstrate use of the custom unit and property models.

In [4]:
from pyomo.environ import ConcreteModel

from idaes.core import FlowsheetBlock

from simple_prop_model import PropParameterBlock
from simple_unit_model import Filtration

In [5]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = PropParameterBlock()
m.fs.unit = Filtration(property_package=m.fs.properties)